In [1]:
import json

import requests

def set_lamp_brightness(mode):
    # URL for the Thing Description (base URL provided in the TD itself)
    td_url = "http://10.2.2.33:1880/r402/theglobe"

    try:
        # Send a GET request to retrieve the Thing Description
        response = requests.get(td_url)

        if not response.json()["on"]:
            switch_lamp_on_state()
            return None

        # get the current brightness of the lamp
        curr_brighness = response.json()["brightness"]
        lamp_brighness = curr_brighness

        # if mode is increase, increase the brightness by 10 if it is less than 100
        if mode == "up":
            if curr_brighness <= 80:
                lamp_brighness += 20
            else:
                lamp_brighness = 100
        # if mode is decrease, decrease the brightness by 10 if it is greater than 0
        elif mode == "down":
            if curr_brighness >= 20:
                lamp_brighness -= 20
            else:
                lamp_brighness = 0

        # Check if the request was successful
        if response.status_code == 200:
            # Define the payload to turn the lamp off
            payload = {
                "brightness": lamp_brighness
            }

            # Send the PUT request to change the state of the lamp
            try:
                response = requests.put(td_url, json=payload)
                if response.status_code == 200:
                    print(f"Lamp brightness has been set to {lamp_brighness}.")
                else:
                    print(f"Failed to turn off the lamp. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
        else:
            print(f"Failed to retrieve Thing Description. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

def set_lamp_color(mode):
    # URL for the Thing Description (base URL provided in the TD itself)
    td_url = "http://10.2.2.33:1880/r402/theglobe"
    colors = ["red", "green", "gold", "violet"]

    try:
        # Send a GET request to retrieve the Thing Description
        response = requests.get(td_url)

        if not response.json()["on"]:
            switch_lamp_on_state()
            return None

        # get the current color of the lamp
        curr_color = response.json()["color"]
        print(curr_color)
        if curr_color in colors:
            curr_index = colors.index(curr_color)
        else:
            curr_index = 0

        # if mode is increase, increase the brightness by 10 if it is less than 100
        if mode == "left":
            if curr_index == 0:
                curr_index = len(colors) - 1
            else:
                curr_index = curr_index - 1
        # if mode is decrease, decrease the brightness by 10 if it is greater than 0
        elif mode == "right":
            if curr_index == len(colors) - 1:
                curr_index = 0
            else:
                curr_index = curr_index + 1

        # Check if the request was successful
        if response.status_code == 200:
            # Define the payload to turn the lamp off
            payload = {
                "color": colors[curr_index]
            }

            # Send the PUT request to change the state of the lamp
            try:
                response = requests.put(td_url, json=payload)
                if response.status_code == 200:
                    print(f"Lamp color has been set to {colors[curr_index]}.")
                else:
                    print(f"Failed to set lamp color. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
        else:
            print(f"Failed to retrieve Thing Description. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

def switch_lamp_on_state():
    td_url = "http://10.2.2.33:1880/r402/theglobe"
    try:
        response = requests.get(td_url)
        if response.status_code == 200:
            payload = {
                "on": True if response.json()["on"] == False else False
            }
            response = requests.put(td_url, json=payload)
            if response.status_code == 200:
                print("Lamp has been turned on.")
            else:
                print(f"Failed to turn on the lamp. Status code: {response.status_code}")
        else:
            print(f"Failed to retrieve Thing Description. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")



In [2]:
import pprint
import time

import zmq
from sympy.physics.units import current

ctx = zmq.Context()
# The REQ talks to Pupil remote and receives the session unique IPC SUB PORT
socket = ctx.socket(zmq.REQ)

ip = 'localhost'
port = 50020

socket.connect(f'tcp://{ip}:{port}')

# Request 'SUB_PORT' for reading data
socket.send_string('SUB_PORT')
sub_port = socket.recv_string()

# Request 'PUB_PORT' for writing data
socket.send_string('PUB_PORT')
pub_port = socket.recv_string()

socket.close()

In [3]:
def create_socket(ctx_c, ip_c, port_sub, topics):
    sub = ctx_c.socket(zmq.SUB)
    sub.connect(f'tcp://{ip_c}:{port_sub}')
    for topic in topics:
        sub.subscribe(topic)
    return sub

In [4]:
import requests

def move_tractorbot(mode):
    action_url = "http://10.2.2.183/actions/wheelcontrol"

    axis = 0
    speed = 0
    duration = 0

    if mode == "left":
        axis = 0
        speed = 3
        duration = 1000
    elif mode == "right":
        axis = 0
        speed = -3
        duration = 1000
    elif mode == "up":
        axis = 1
        speed = 3
        duration = 500
    elif mode == "down":
        axis = 2
        speed = 3
        duration = 500

    payload = {
        "axis": axis,
        "speed": speed,
        "duration": duration
    }

    try:
        headers = {"Content-Type": "application/json"}
        response = requests.post(action_url, json=payload, headers=headers)
        if response.status_code == 200:
            print("Action successfully invoked:", response.json())
        else:
            print(f"Failed to invoke action. Status code: {response.status_code}, Response: {response.text}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")


In [ ]:
import msgpack

topics = ["gestures"] #, "objects"]
socket_sub = create_socket(ctx, ip, sub_port, topics)

while True:
    topic = socket_sub.recv_string()
    payload = socket_sub.recv()
    msg = msgpack.unpackb(payload, raw=False)
    if topic == "gestures":
        pprint.pprint(msg)
        if "HueLamp" in msg["object"]:
            if msg["direction"] == "left" or msg["direction"] == "right":
                set_lamp_color(msg["direction"])
            elif msg["direction"] == "up" or msg["direction"] == "down":
                set_lamp_brightness(msg["direction"])

        elif "Tractorbot" in msg["object"]:
            move_tractorbot(msg["direction"])